In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy
from PIL import Image

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#커스텀 신경망 클래스 선언

class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.f = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3, device = device), #(3,32,32)>(32,30,30)
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3, device = device), #(32,30,30)>(32,28,28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(32,28,28)>(16,14,14)
            nn.Dropout(0.25),

            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3, device = device), #(32,14,14)>(64,12,12)
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3, device = device), #(64,12,12)>(64,10,10)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(64,10,10)>(64,5,5)
            nn.Dropout(0.25),

            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3, device = device), #(64,5,5)>(128,3,3)
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3, device = device), #(128,3,3)>(256,1,1)
            nn.ReLU(),
            nn.Dropout(0.25),
        )
        self.g = nn.Sequential(
            nn.Linear(256, 128, device = device),
            nn.ReLU(),
            nn.Linear(128, 64, device = device),
            nn.ReLU(),
            nn.Linear(64, 10, device = device),
        )
    def forward(self, x) :
        if len(x.shape) == 3 :
            x = x.unsqueeze(dim = 0)
        x = self.f(x)
        x = x.reshape(x.shape[0], -1)
        x = self.g(x)
        return x

In [3]:
#커스텀 신경망 불러오기

F = torch.load("CIFAR10.pt", weights_only=False, map_location=torch.device(device))

In [4]:
#테스트 사진 불러오기
test = datasets.CIFAR10(root = "/data", train = False, transform=transforms.ToTensor(), download=True)

test_data = DataLoader(test, batch_size=1000, shuffle = False)

In [5]:
cnt = 0
idx = 0
err = []

for x, t in test_data :
    x = x.to(device)
    t = t.to(device)
    y = F(x)
    for i in range(len(x)) :
        if torch.argmax(y[i]) == t[i] :
            cnt += 1
        else :
            err.append(idx)
        idx += 1
            
print(cnt / len(test))
print(err)

0.8084
[3, 10, 12, 24, 32, 35, 36, 37, 47, 52, 56, 57, 58, 59, 61, 70, 76, 78, 81, 85, 99, 118, 121, 128, 139, 147, 148, 158, 160, 164, 169, 178, 188, 193, 210, 211, 224, 228, 232, 239, 245, 247, 249, 255, 257, 258, 259, 263, 264, 265, 277, 284, 287, 295, 306, 308, 309, 312, 313, 321, 323, 352, 356, 368, 370, 377, 378, 384, 387, 399, 416, 418, 422, 427, 428, 438, 439, 450, 453, 455, 456, 459, 470, 477, 480, 483, 488, 518, 525, 531, 532, 537, 538, 539, 541, 548, 551, 553, 563, 565, 567, 577, 587, 597, 598, 621, 629, 630, 647, 651, 652, 655, 657, 663, 665, 671, 675, 676, 683, 685, 689, 692, 710, 725, 731, 734, 735, 739, 740, 744, 746, 749, 761, 766, 767, 776, 779, 792, 793, 796, 799, 802, 810, 811, 828, 830, 831, 836, 845, 850, 853, 862, 877, 882, 883, 887, 888, 893, 898, 900, 910, 912, 914, 916, 924, 933, 935, 943, 957, 965, 966, 967, 972, 982, 988, 993, 1019, 1032, 1034, 1035, 1040, 1042, 1043, 1047, 1050, 1053, 1065, 1070, 1072, 1074, 1082, 1088, 1090, 1095, 1100, 1112, 1118, 1124, 11

In [53]:
with open("list.txt", mode = "r") as f:
    li = f.readlines()

sample = test[4][0]
sample = numpy.transpose(sample.numpy() * 255, (1,2,0))
sample = sample.astype(numpy.uint8)

img = Image.fromarray(sample)
y = F(test[4][0].to(device))

print(f"y : {li[y.argmax()]}")
print(f"t : {li[test[4][1]]}")

img.show()

y : Frog

t : Frog



In [63]:
img = Image.open("dog_2.jpg")
img = img.convert("RGB").resize((32,32))
arr = numpy.transpose(numpy.array(img),(2,0,1)) / 255
ten = torch.tensor(arr, dtype = torch.float, device = device)
print(ten)
print(ten.shape)

y = F(ten)
print(f"y : {li[y.argmax()]}")
img.show()

tensor([[[0.2824, 0.2784, 0.2824,  ..., 0.2510, 0.2275, 0.2627],
         [0.2588, 0.2471, 0.2588,  ..., 0.3216, 0.2314, 0.2784],
         [0.3137, 0.3098, 0.2941,  ..., 0.3725, 0.3255, 0.3294],
         ...,
         [0.2745, 0.2902, 0.2510,  ..., 0.2745, 0.2078, 0.1961],
         [0.2588, 0.2784, 0.2275,  ..., 0.2275, 0.2000, 0.2157],
         [0.2275, 0.2431, 0.2314,  ..., 0.2431, 0.2980, 0.2118]],

        [[0.3882, 0.3922, 0.3922,  ..., 0.3843, 0.3569, 0.3490],
         [0.3961, 0.3843, 0.4000,  ..., 0.4157, 0.3569, 0.3882],
         [0.4275, 0.4235, 0.4157,  ..., 0.4627, 0.4314, 0.4353],
         ...,
         [0.3412, 0.3725, 0.3294,  ..., 0.3804, 0.3216, 0.2706],
         [0.3373, 0.3647, 0.3216,  ..., 0.3255, 0.3020, 0.3020],
         [0.3255, 0.3137, 0.3098,  ..., 0.3608, 0.4000, 0.2902]],

        [[0.2039, 0.1922, 0.2039,  ..., 0.1804, 0.1647, 0.1765],
         [0.1804, 0.1686, 0.1412,  ..., 0.2275, 0.1725, 0.1961],
         [0.1961, 0.1882, 0.1765,  ..., 0.2627, 0.2667, 0.